<a href="https://colab.research.google.com/github/sarahfirouzabadi/Summer-2025-ECE-597-Group5/blob/main/Summer_2025_ECE_597_Group5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import Word, TextBlob
from scipy.sparse import hstack, csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, auc, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
TFIDV=TfidfVectorizer()

In [2]:
CEAS=pd.read_csv('/CEAS_08.csv')
CEAS.head()

FileNotFoundError: [Errno 2] No such file or directory: '/CEAS_08.csv'

In [ ]:
label_counts = CEAS['label'].value_counts().sort_index()
num_ham = label_counts.get(0, 0)
num_spam = label_counts.get(1, 0)

print(f"CEAS:\nHam: {num_ham}\nSpam: {num_spam}")

In [ ]:
email_counts = CEAS['label'].value_counts().rename({0: 'Normal', 1: 'Spam'})

email_counts.plot.pie(
    autopct='%1.1f%%',
    colors=['#66b3ff','#ff6666'],
    startangle=90,
    explode=(0, 0.1),
    shadow=True
)

plt.title('Percentage of Spam vs Normal Emails')
plt.ylabel('')  # remove default y-axis label

plt.show()

In [ ]:
SpamAssasin = pd.read_csv('/SpamAssasin.csv')
SpamAssasin.head()

In [ ]:
label_counts = SpamAssasin['label'].value_counts().sort_index()
num_ham = label_counts.get(0, 0)
num_spam = label_counts.get(1, 0)

print(f"SpamAssasin:\nHam: {num_ham}\nSpam: {num_spam}")

In [ ]:
email_counts = SpamAssasin['label'].value_counts().rename({0: 'Normal', 1: 'Spam'})

email_counts.plot.pie(
    autopct='%1.1f%%',
    colors=['#66b3ff','#ff6666'],
    startangle=90,
    explode=(0, 0.1),
    shadow=True
)

plt.title('Percentage of Spam vs Normal Emails')
plt.ylabel('')

plt.show()

In [ ]:
Nazario = pd.read_csv('/Nazario.csv')
Nazario.head()

In [ ]:
label_counts = Nazario['label'].value_counts().sort_index()
num_ham = label_counts.get(0, 0)
num_spam = label_counts.get(1, 0)

print(f"Nazario:\nHam: {num_ham}\nSpam: {num_spam}")

In [ ]:
email_counts = Nazario['label'].value_counts().rename({0: 'Normal', 1: 'Spam'})

email_counts.plot.pie(
    autopct='%1.1f%%',
    colors=['#ff6666'],
    startangle=90,
    shadow=True
)

plt.title('Percentage of Spam vs Normal Emails')
plt.ylabel('')

plt.show()

In [ ]:
Nigerian_Fraud = pd.read_csv('/Nigerian_Fraud.csv')
Nigerian_Fraud.head()

In [ ]:
label_counts = Nigerian_Fraud['label'].value_counts().sort_index()
num_ham = label_counts.get(0, 0)
num_spam = label_counts.get(1, 0)

print(f"Nigerian_Fraud:\nHam: {num_ham}\nSpam: {num_spam}")

In [ ]:
email_counts = Nigerian_Fraud['label'].value_counts().rename({0: 'Normal', 1: 'Spam'})

email_counts.plot.pie(
    autopct='%1.1f%%',
    colors=['#ff6666'],
    startangle=90,
    shadow=True
)

plt.title('Percentage of Spam vs Normal Emails')
plt.ylabel('')

plt.show()

In [ ]:
# Combine datasets and removing unnecessary coloumns
combined_df = pd.concat([
    CEAS[['sender', 'subject', 'body', 'label']],
    SpamAssasin[['sender', 'subject', 'body', 'label']],
    Nazario[['sender', 'subject', 'body', 'label']],
    Nigerian_Fraud[['sender', 'subject', 'body', 'label']]
], ignore_index=True)

print(combined_df.head())
print(f"Combined DataFrame shape: {combined_df.shape}")

In [ ]:
# Remove duplicates
combined_df.drop_duplicates(subset=['sender', 'subject', 'body'], inplace=True)

# Missing values
print(combined_df.isnull().sum())

# Drop rows with missing sender, subject, or body
combined_df.dropna(subset=['sender', 'subject', 'body'], inplace=True)

combined_df.reset_index(drop=True, inplace=True)
print(f'Cleaned dataset shape: {combined_df.shape}')

In [ ]:
# Count spam and normal emails
email_counts = combined_df['label'].value_counts().rename({0: 'Normal', 1: 'Spam'})

# Plot a pie chart
email_counts.plot.pie(
    autopct='%1.1f%%',
    colors=['#66b3ff','#ff6666'],
    startangle=90,
    explode=(0, 0.1),
    shadow=True
)

plt.title('Percentage of Spam vs Normal Emails')
plt.ylabel('')  # remove default y-axis label

plt.show()

Down sampling (Reducing spam size)

In [ ]:
spam = combined_df[combined_df.label == 1]
normal = combined_df[combined_df.label == 0]

# Downsample spam class to match normal class size
spam_downsampled = resample(spam,
                            replace=False,
                            n_samples=len(normal),
                            random_state=42)

# Combine them all again
balanced_df = pd.concat([spam_downsampled, normal])

# Shuffle rows
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


print(balanced_df['label'].value_counts())

In [ ]:
balanced_df.info()
balanced_df.head()

In [ ]:
sns.countplot(x='label', data=balanced_df)
plt.title('Spam vs Normal Emails')
plt.xticks([0, 1], ['Normal', 'Spam'])
plt.show()

In [ ]:
balanced_df['email_length'] = balanced_df['body'].fillna('').str.len()

filtered_df = balanced_df[balanced_df['email_length'] < 5000]

# Plot the filtered data
sns.histplot(data=filtered_df, x='email_length', hue='label', bins=50, kde=True)
plt.title('Distribution of Email Lengths (filtered under 5000 characters)')
plt.xlabel('Email Length (characters)')
plt.show()

In [ ]:
spam_words = ' '.join(balanced_df[balanced_df['label']==1]['body'].dropna())
wordcloud_spam = WordCloud(width=800, height=400, max_words=100).generate(spam_words)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud_spam, interpolation='bilinear')
plt.title('Spam Emails - Frequent Words')
plt.axis('off')
plt.show()

In [ ]:
# Download stopwords once
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize counter
word_counter = Counter()

# Loop through spam emails line by line (avoids joining the full string)
for text in balanced_df[balanced_df['label'] == 1]['body'].dropna():
    # Lowercase, remove punctuation/numbers, tokenize
    words = re.findall(r'\b[a-z]+\b', text.lower())

    # Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]

    # Update counter
    word_counter.update(filtered_words)

# Show top 50
print("Top 50 frequent words in spam emails:")
for word, freq in word_counter.most_common(50):
    print(f"{word}: {freq}")

In [ ]:
def tokenize(text):
    text = str(text).lower()
    words = re.findall(r'\b[a-z]+\b', text)  # only alphabetic words
    return [w for w in words if w not in stop_words]

# Apply tokenization
balanced_df['tokens'] = balanced_df['body'].apply(tokenize)

# Separate phishing and normal
phishing_tokens = balanced_df[balanced_df['label'] == 1]['tokens']
normal_tokens = balanced_df[balanced_df['label'] == 0]['tokens']

# Average words
avg_words_phishing = phishing_tokens.apply(len).mean()
avg_words_normal = normal_tokens.apply(len).mean()

# Flatten token lists
flat_phishing = [word for tokens in phishing_tokens for word in tokens]
flat_normal = [word for tokens in normal_tokens for word in tokens]

# Unique word counts
unique_phishing = len(set(flat_phishing))
unique_normal = len(set(flat_normal))

# Most indicative phishing words (appearing more in phishing than in normal)
phishing_counter = Counter(flat_phishing)
normal_counter = Counter(flat_normal)

indicative_phishing = {
    word: freq for word, freq in phishing_counter.items()
    if freq > normal_counter.get(word, 0)
}

top_indicative = Counter(indicative_phishing).most_common(20)

print(f"Average words per phishing email: {avg_words_phishing:.2f}")
print(f"Total unique words in phishing emails: {unique_phishing}")

print(f"Average words per normal email: {avg_words_normal:.2f}")
print(f"Total unique words in normal emails: {unique_normal}")

print("\nTop 20 words most indicative of phishing:")
for word, freq in top_indicative:
    print(f"{word}: {freq}")

In [ ]:
# Average words data
avg_words = {'Phishing': 120.63, 'Normal': 215.75}

plt.figure(figsize=(6, 4))
sns.barplot(x=list(avg_words.keys()), y=list(avg_words.values()))
plt.title('Average Words per Email')
plt.ylabel('Average Word Count')
plt.ylim(0, max(avg_words.values()) + 50)
plt.show()

In [ ]:
# Unique words data
unique_words = {'Phishing': 81119, 'Normal': 105622}

plt.figure(figsize=(6, 4))
sns.barplot(x=list(unique_words.keys()), y=list(unique_words.values()))
plt.title('Total Unique Words')
plt.ylabel('Unique Word Count')
plt.ylim(0, max(unique_words.values()) * 1.1)
plt.show()


Subject

In [ ]:
balanced_df['subject_length'] = balanced_df['subject'].fillna('').apply(len)

plt.figure(figsize=(6,4))
sns.histplot(data=balanced_df, x='subject_length', hue='label', bins=50, kde=True)
plt.title('Distribution of Subject Lengths')
plt.xlabel('Subject Length (characters)')
plt.ylabel('Email Count')
plt.show()


In [ ]:
# Filter out very long subjects
filtered_subject_df = balanced_df[balanced_df['subject_length'] < 200]

plt.figure(figsize=(6,4))
sns.histplot(data=filtered_subject_df, x='subject_length', hue='label', bins=50, kde=True)
plt.title('Distribution of Subject Lengths (Filtered < 200 chars)')
plt.xlabel('Subject Length (characters)')
plt.ylabel('Email Count')
plt.show()

In [ ]:
avg_len_spam = balanced_df[balanced_df['label'] == 1]['subject_length'].mean()
avg_len_normal = balanced_df[balanced_df['label'] == 0]['subject_length'].mean()

print(f"Average subject length - Phishing: {avg_len_spam:.2f}")
print(f"Average subject length - Normal: {avg_len_normal:.2f}")

In [ ]:
sns.barplot(x=['Phishing', 'Normal'], y=[avg_len_spam, avg_len_normal])
plt.title('Average Subject Length')
plt.ylabel('Characters')
plt.show()

In [ ]:
stop_words = set(stopwords.words('english'))

def clean_subject(text):
    words = re.findall(r'\b[a-z]+\b', str(text).lower())
    return [w for w in words if w not in stop_words]

# Tokenize spam and normal subjects
spam_subject_words = balanced_df[balanced_df['label'] == 1]['subject'].dropna().apply(clean_subject)
normal_subject_words = balanced_df[balanced_df['label'] == 0]['subject'].dropna().apply(clean_subject)

# Flatten
spam_subject_flat = [word for words in spam_subject_words for word in words]
normal_subject_flat = [word for words in normal_subject_words for word in words]

# Count top words
spam_top_subject = Counter(spam_subject_flat).most_common(15)
normal_top_subject = Counter(normal_subject_flat).most_common(15)

# For spam
df_spam_subject = pd.DataFrame(spam_top_subject, columns=['Word', 'Count'])
sns.barplot(data=df_spam_subject, y='Word', x='Count')
plt.title('Top Words in Phishing Email Subjects')
plt.show()

# For normal
df_normal_subject = pd.DataFrame(normal_top_subject, columns=['Word', 'Count'])
sns.barplot(data=df_normal_subject, y='Word', x='Count')
plt.title('Top Words in Normal Email Subjects')
plt.show()


Extracting the Domain

In [ ]:
balanced_df["sender_2"]=balanced_df['sender'].str.split('@').str[1]

In [ ]:
balanced_df["sender_2"].value_counts()

Data cleaning

In [ ]:
def clean_and_tokenize(text):
    text = str(text).lower()                             # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)                 # Remove punctuation and numbers
    tokens = text.split()                                # Tokenize by whitespace
    tokens = [t for t in tokens if t not in stop_words]  # Remove stopwords
    return tokens


balanced_df['body_tokens'] = balanced_df['body'].fillna('').apply(clean_and_tokenize)
balanced_df['subject_tokens'] = balanced_df['subject'].fillna('').apply(clean_and_tokenize)

print("Sample cleaned body:", balanced_df['body_tokens'].iloc[0])
print("Sample cleaned subject:", balanced_df['subject_tokens'].iloc[0])

Feature engineering

In [ ]:
# Email body length
balanced_df['email_length'] = balanced_df['body'].fillna('').str.len()

# Subject length
balanced_df['subject_length'] = balanced_df['subject'].fillna('').str.len()

# Number of words in the body
balanced_df['body_word_count'] = balanced_df['body_tokens'].apply(len)

# Number of words in the subject
balanced_df['subject_word_count'] = balanced_df['subject_tokens'].apply(len)

# Number of URLs
url_pattern = r'(https?://[^\s]+)' # Define a URL regex pattern
balanced_df['num_urls'] = balanced_df['body'].fillna('').apply(lambda x: len(re.findall(url_pattern, x)))

# Sender domain
balanced_df['sender_domain'] = balanced_df['sender'].fillna('').str.extract(r'@([\w\.-]+)')

# Preview engineered features
balanced_df[['email_length', 'subject_length', 'body_word_count', 'subject_word_count',
             'num_urls', 'sender_domain']].head()


TF-IDF

In [ ]:
# Join tokens back to strings
balanced_df['body_clean'] = balanced_df['body_tokens'].apply(lambda x: ' '.join(x))
balanced_df['subject_clean'] = balanced_df['subject_tokens'].apply(lambda x: ' '.join(x))
balanced_df['sender_domain_clean'] = balanced_df['sender_domain'].fillna('')

# body
tfidf_body = TfidfVectorizer(stop_words='english', max_features=3000)
X_body = tfidf_body.fit_transform(balanced_df['body_clean'])

# subject
tfidf_subject = TfidfVectorizer(stop_words='english', max_features=1000)
X_subject = tfidf_subject.fit_transform(balanced_df['subject_clean'])

# sender domain
tfidf_sender = TfidfVectorizer()
X_sender = tfidf_sender.fit_transform(balanced_df['sender_domain_clean'])

print("TF-IDF shape (body):", X_body.shape)
print("TF-IDF shape (subject):", X_subject.shape)
print("TF-IDF shape (sender):", X_sender.shape)

Train Logistic Regression and Random Forest

In [ ]:
y = balanced_df['label']

numeric_cols = [
    'email_length',
    'subject_length',
    'body_word_count',
    'subject_word_count',
    'num_urls'
]
X_numeric_raw = balanced_df[numeric_cols].fillna(0)

scaler = StandardScaler()
X_numeric = scaler.fit_transform(X_numeric_raw)


X_all = hstack([
    X_body,
    X_subject,
    X_sender,
    csr_matrix(X_numeric)
])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

log_model_cv = LogisticRegression(max_iter=1000)

# Evaluate AUC across 5 folds
cv_auc_scores = cross_val_score(log_model_cv, X_all, y, cv=cv, scoring='roc_auc')

print("Logistic Regression Cross-Validation AUC scores:", cv_auc_scores)
print(f"Mean AUC: {cv_auc_scores.mean():.4f}, Std: {cv_auc_scores.std():.4f}")


In [ ]:
# Logistic Regression:

log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

# Evaluating
y_pred_log = log_model.predict(X_test)
y_proba_log = log_model.predict_proba(X_test)[:, 1]

print("Logistic Regression:")
print(classification_report(y_test, y_pred_log))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba_log))

In [ ]:
# ROC for Logistic Regression
fpr_log, tpr_log, _ = roc_curve(y_test, y_proba_log)
auc_log = roc_auc_score(y_test, y_proba_log)

plt.figure(figsize=(6, 5))
plt.plot(fpr_log, tpr_log, label=f'AUC = {auc_log:.4f}')
plt.plot([0, 1], [0, 1], 'k--')  # diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Confusion matrix for Logistic Regression
cm_log = confusion_matrix(y_test, y_pred_log)

disp_log = ConfusionMatrixDisplay(confusion_matrix=cm_log, display_labels=["Normal", "Spam"])
disp_log.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Logistic Regression")
plt.grid(False)
plt.show()

In [ ]:
# Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,       # number of trees
    random_state=42,        # ensures reproducibility
    n_jobs=-1               # uses all CPU cores for training
)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba_rf))

In [ ]:
# ROC for Random Forest
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)
auc_rf = roc_auc_score(y_test, y_proba_rf)

plt.figure(figsize=(6, 5))
plt.plot(fpr_rf, tpr_rf, label=f'AUC = {auc_rf:.4f}', color='green')
plt.plot([0, 1], [0, 1], 'k--')  # diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Confusion matrix for Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)

disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=["Normal", "Spam"])
disp_rf.plot(cmap='Greens', values_format='d')
plt.title("Confusion Matrix - Random Forest")
plt.grid(False)
plt.show()

In [ ]:
comparison_data = {
    "Model": ["Logistic Regression", "Random Forest"],
    "Accuracy": [log_model.score(X_test, y_test), rf_model.score(X_test, y_test)],
    "ROC_AUC": [roc_auc_score(y_test, y_proba_log), roc_auc_score(y_test, y_proba_rf)],
}
pd.DataFrame(comparison_data)

In [ ]:
# Get numeric feature importances from Random Forest
importances = rf_model.feature_importances_[-X_numeric.shape[1]:]  # last features are numeric
feature_names = numeric_cols

# Plot
plt.figure(figsize=(6, 4))
sns.barplot(x=importances, y=feature_names)
plt.title("Numeric Feature Importances (Random Forest)")
plt.xlabel("Importance Score")
plt.tight_layout()
plt.show()

In [ ]:
# Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train.toarray(), y_train)  # Convert sparse matrix to dense

y_pred_nb = nb_model.predict(X_test.toarray())
y_proba_nb = nb_model.predict_proba(X_test.toarray())[:, 1]

print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba_nb))

# ROC
fpr_nb, tpr_nb, _ = roc_curve(y_test, y_proba_nb)
auc_nb = roc_auc_score(y_test, y_proba_nb)

plt.figure(figsize=(6, 5))
plt.plot(fpr_nb, tpr_nb, label=f'AUC = {auc_nb:.4f}', color='blue')
plt.plot([0, 1], [0, 1], 'k--')  # diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Naive Bayes')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Confusion matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)

disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb, display_labels=["Normal", "Spam"])
disp_nb.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix - Naive Bayes")
plt.grid(False)
plt.show()
